#### Return the order in which the vars were picked by the SFS methods

Also extract whether the order is robust across subsets or if we remove condensate-free cells.

In [42]:
import os
import json
import numpy as np

In [4]:
parent_pwd = '/home/b/b309170/workspace_icon-ml/symbolic_regression/'

**On the entire dataset**

In [89]:
# Produced by /home/b/b309170/workspace_icon-ml/symbolic_regression/baselines/linear_v2.ipynb
p1_file = parent_pwd + 'baselines/linear_results_v2/dyamond_data/linear_function_fit_cl_area_with_derivatives.txt'
# Produced by /home/b/b309170/workspace_icon-ml/symbolic_regression/baselines/multivariate_polynomial_fit_v2.ipynb
p2_file = parent_pwd + 'baselines/polynomial_results_v2/dyamond_data/normalized_data/without_regimes/polynomial_fit_cl_area_with_derivatives_degree_2.json'
p3_file = parent_pwd + 'baselines/polynomial_results_v2/dyamond_data/normalized_data/without_regimes/polynomial_fit_cl_area_with_derivatives_degree_3.json'
# Produced by /home/b/b309170/workspace_icon-ml/symbolic_regression/finding_symmetries/seq_feature_selector_dyamond_nns.ipynb
nn_file = parent_pwd + 'finding_symmetries/seq_feature_selector_dyamond_nns/'

Linear model

In [47]:
features = {}
with open(p1_file, 'r') as file:
    i = 0
    print_bool = False
    while True:
        line = file.readline()

        if 'The R2-score on the training data' in line:
            print_bool = False
            
        if print_bool and '*' in line:
            features[i][j] = line.split('*')[1][:-1]
            j += 1
            
        if 'Linear function fit:' in line:
            i += 1
            features[i] = {}
            print_bool = True
            j = 0
            
        if i == 10:
            break

In [73]:
print(set(features[1].values()))
for k in range(1, 9):
    print(set(features[k+1].values()).symmetric_difference(set(features[k].values())))

{'rh'}
{'ta'}
{'cli'}
{'clw'}
{'hus'}
{'U'}
{'pa_zz'}
{'rh_z'}
{'hus_zz'}


Polynomial, degree 2

In [71]:
with open(p2_file,'r') as file:
    d = json.load(file)
    print(set(list(d['Number of variables %d'%1].keys())[:-7]))
    for k in range(1, 9):
        features_old = list(d['Number of variables %d'%k].keys())[:-7]
        features_new = list(d['Number of variables %d'%(k+1)].keys())[:-7]
        print(set(features_new).symmetric_difference(features_old))

{'rh'}
{'ta'}
{'clw cli'}
{'rh^2'}
{'ta rh'}
{'pa hus_z'}
{'hus rh'}
{'pa U_z'}
{'clw_z rh_z'}


Polynomial, degree 3

In [72]:
with open(p3_file,'r') as file:
    d = json.load(file)
    print(set(list(d['Number of variables %d'%1].keys())[:-7]))
    for k in range(1, 9):
        features_old = list(d['Number of variables %d'%k].keys())[:-7]
        features_new = list(d['Number of variables %d'%(k+1)].keys())[:-7]
        print(set(features_new).symmetric_difference(features_old))

{'rh'}
{'ta'}
{'clw ta^2'}
{'cli'}
{'rh^2'}
{'ta rh'}
{'cli hus_z'}
{'hus rh^2'}
{'ta rh^2'}


Neural Networks (three different subsets)

In [95]:
with open(nn_file + 'seq_feat_selector_training_data_cl_area_17_15.json', 'r') as file:
    d = json.load(file)
    print(set(d['features_%d'%1]))
    for k in range(1, 9):
        features_old = set(d['features_%d'%k])
        features_new = set(d['features_%d'%(k+1)])
        print(set(features_new).symmetric_difference(features_old))

{'rh'}
{'cli'}
{'clw'}
{'ta'}
{'rh_z'}
{'pa_z'}
{'pa_zz'}
{'rh_zz'}
{'ta_z'}


In [96]:
with open(nn_file + 'seq_feat_selector_training_data_cl_area_17_17.json', 'r') as file:
    d = json.load(file)
    print(set(d['features_%d'%1]))
    for k in range(1, 9):
        features_old = set(d['features_%d'%k])
        features_new = set(d['features_%d'%(k+1)])
        print(set(features_new).symmetric_difference(features_old))

{'rh'}
{'cli'}
{'clw'}
{'rh_z'}
{'ta'}
{'pa_zz'}
{'pa_z'}
{'rh_zz'}
{'hus'}


In [97]:
with open(nn_file + 'seq_feat_selector_training_data_cl_area_17_19.json', 'r') as file:
    d = json.load(file)
    print(set(d['features_%d'%1]))
    for k in range(1, 9):
        features_old = set(d['features_%d'%k])
        features_new = set(d['features_%d'%(k+1)])
        print(set(features_new).symmetric_difference(features_old))

{'rh'}
{'cli'}
{'clw'}
{'rh_z'}
{'ta'}
{'pa_zz'}
{'rh_zz'}
{'zg'}
{'ta_z'}


**Features that we actually picked for the SFS NNs** <br>

**--> The ones from seq_feat_selector_training_data_cl_area_17_15.json**

In [101]:
features = {}
path_sfs_nns = '/home/b/b309170/workspace_icon-ml/cloud_cover_parameterization/neighborhood_based_on_seq_feat_sel_DYAMOND/saved_models/'
for k in range(1, 11):
    with open(path_sfs_nns + 'neighborhood_based_sfs_cl_area_no_features_%d.txt'%k, 'r') as file:
        print_bool = False
        while True:
            line = file.readline()

            if 'The (order of) input variables' in line:
                print_bool = False
                break

            if print_bool:
                features[k] = line

            if 'Input and output variables' in line:
                features[k] = {}
                print_bool = True

In [117]:
print(features[1].split('\'')[1])
for k in range(1, 10):
    features_old = set(features[k].split('\'')[1:-1])
    features_new = set(features[k+1].split('\'')[1:-1])
    print(set(features_new).symmetric_difference(features_old))

rh
{'cli'}
{'clw'}
{'ta'}
{'rh_z'}
{'pa_z'}
{'pa_zz'}
{'rh_zz'}
{'ta_z'}
{'ps'}


**With the condensate-free cells removed**

**--> These were the features I picked for the Pareto plot polynomials!**

In [74]:
p1_file = parent_pwd + 'baselines/polynomial_results_v2/dyamond_data/normalized_data/data_driven_regimes/polynomial_fit_cl_area_with_derivatives_degree_1_dt_basis_False_no_of_regimes_2_regime_1.json'
p2_file = parent_pwd + 'baselines/polynomial_results_v2/dyamond_data/normalized_data/data_driven_regimes/polynomial_fit_cl_area_with_derivatives_degree_2_dt_basis_False_no_of_regimes_2_regime_1.json'
p3_file = parent_pwd + 'baselines/polynomial_results_v2/dyamond_data/normalized_data/data_driven_regimes/polynomial_fit_cl_area_with_derivatives_degree_3_dt_basis_False_no_of_regimes_2_regime_1.json'

Linear model

In [80]:
with open(p1_file,'r') as file:
    d = json.load(file)
    print(set(list(d['Number of variables %d'%1].keys())[:-5]))
    for k in range(1, 9):
        features_old = list(d['Number of variables %d'%k].keys())[:-5]
        features_new = list(d['Number of variables %d'%(k+1)].keys())[:-5]
        print(set(features_new).symmetric_difference(features_old))

{'rh'}
{'ta'}
{'rh_z'}
{'cli'}
{'pa_zz'}
{'clw'}
{'U'}
{'clw_zz'}
{'hus_z'}


In [81]:
with open(p2_file,'r') as file:
    d = json.load(file)
    print(set(list(d['Number of variables %d'%1].keys())[:-5]))
    for k in range(1, 9):
        features_old = list(d['Number of variables %d'%k].keys())[:-5]
        features_new = list(d['Number of variables %d'%(k+1)].keys())[:-5]
        print(set(features_new).symmetric_difference(features_old))

{'rh'}
{'ta'}
{'clw cli'}
{'rh rh_z'}
{'ta rh_z'}
{'hus rh'}
{'ta rh'}
{'rh^2'}
{'hus_z'}


In [82]:
with open(p3_file,'r') as file:
    d = json.load(file)
    print(set(list(d['Number of variables %d'%1].keys())[:-5]))
    for k in range(1, 9):
        features_old = list(d['Number of variables %d'%k].keys())[:-5]
        features_new = list(d['Number of variables %d'%(k+1)].keys())[:-5]
        print(set(features_new).symmetric_difference(features_old))

{'rh'}
{'ta'}
{'clw cli'}
{'ta^2 rh'}
{'rh^2'}
{'ta^2'}
{'rh_z'}
{'pa_zz pa^2'}
{'rh rh_z'}


### Averaged SFS order of variables over 10 seeds

In [57]:
# Produced by /home/b/b309170/workspace_icon-ml/symbolic_regression/baselines/multivariate_polynomial_fit_v2.ipynb
p_folder = parent_pwd + 'baselines/polynomial_results_v2/dyamond_data/normalized_data/data_driven_regimes/more_seeds/'
# Produced by /home/b/b309170/workspace_icon-ml/symbolic_regression/finding_symmetries/seq_feature_selector_dyamond_nns.ipynb
nn_folder = parent_pwd + 'finding_symmetries/seq_feature_selector_dyamond_nns/'
nn_folder_no_cond_free = parent_pwd + 'finding_symmetries/seq_feature_selector_dyamond_nns/no_condensate_free_cells/'

In [95]:
def return_averaged_ranking(folder_abs_path, model_type, max_no_features=10): 
    '''
        folder_abs_path: Folder that contains the relevant files (absolute path)
        model_type: 'nn', 'polynomial_degree_1', 'polynomial_degree_2', 'polynomial_degree_3'
    '''
    var_position = {}
    
    if model_type == 'nn':
        substring = '0.json'
        key_name = 'features_'
    elif model_type.startswith('polynomial'):
        substring = model_type.split('_', 1)[1]
        key_name = 'Number of variables '

    for file_name in os.listdir(folder_abs_path):
        # Open the file if it is one of the 10 seeds
        if substring in file_name:
            with open(folder_abs_path + file_name, 'r') as file:
                d = json.load(file)
                # Either we can append to the list otherwise it needs to be initialized
                if model_type == 'nn':
                    try:
                        var_position[d['%s%d'%(key_name,1)][0]].append(1)
                    except:
                        var_position[d['%s%d'%(key_name,1)][0]] = [1]
                elif model_type.startswith('polynomial'):
                    try:
                        var_position[list(d['%s%d'%(key_name,1)].keys())[:-5][0]].append(1)
                    except:
                        var_position[list(d['%s%d'%(key_name,1)].keys())[:-5][0]] = [1]
                for k in range(1, max_no_features):
                    if model_type == 'nn':
                        features_old = set(d['%s%d'%(key_name,k)])
                        features_new = set(d['%s%d'%(key_name,k+1)])
                    elif model_type.startswith('polynomial'):
                        features_old = set(list(d['%s%d'%(key_name,k)].keys())[:-5])
                        features_new = set(list(d['%s%d'%(key_name,k+1)].keys())[:-5])
                    # The added feature (set containing a single element)
                    added_feature_set = set(features_new).symmetric_difference(features_old)
                    added_feature = added_feature_set.pop()
                    try:
                        var_position[added_feature].append(k+1)
                    except:
                        var_position[added_feature] = [k+1]

    # Set position to max_no_features + 1 if they were not in the top max_no_features for all seeds.
    for key in var_position.keys():
        # There are 10 different seeds
        while len(var_position[key]) < 10:
            var_position[key].append(max_no_features + 1)

    # Compute the average var_positions
    var_position_avrg = {}
    for key in var_position.keys():
        var_position_avrg[key] = np.mean(var_position[key])

    # Sort the variables according to their average position and return the resulting dictionary!
    dict_keys = list(var_position_avrg.keys())
    dict_values = list(var_position_avrg.values())
    sorted_inds = np.argsort(dict_values)
    sorted_dict = {dict_keys[ind]: dict_values[ind] for ind in sorted_inds}
    
    return sorted_dict

NNs, condensate-free cells included

In [96]:
return_averaged_ranking(nn_folder, 'nn', max_no_features=10)

{'rh': 1.0,
 'cli': 2.0,
 'clw': 3.0,
 'ta': 4.1,
 'rh_z': 4.9,
 'pa_zz': 7.1,
 'rh_zz': 8.7,
 'pa_z': 9.0,
 'ps': 9.3,
 'ta_z': 9.5,
 'pa': 9.8,
 'zg': 10.2,
 'cli_z': 10.3,
 'clw_z': 10.5,
 'U_z': 10.8,
 'hus_zz': 10.9,
 'U': 10.9}

NNs, no condensate-free cells

In [97]:
return_averaged_ranking(nn_folder_no_cond_free, 'nn', max_no_features=10)

{'rh': 1.0,
 'cli': 2.0,
 'clw': 3.0,
 'ta': 4.1,
 'rh_z': 4.9,
 'pa_zz': 6.7,
 'pa_z': 8.1,
 'rh_zz': 8.3,
 'ta_z': 10.0,
 'ps': 10.1,
 'hus_zz': 10.1,
 'zg': 10.2,
 'cli_z': 10.5,
 'pa': 10.6,
 'clw_z': 10.7,
 'cli_zz': 10.8,
 'fr_land': 10.9}

Polynomials (no condensate-free cells)

First-order

In [98]:
return_averaged_ranking(p_folder, 'polynomial_degree_1', max_no_features=10)

{'rh': 1.0,
 'ta': 2.0,
 'rh_z': 3.0,
 'cli': 4.3,
 'pa_zz': 4.7,
 'clw': 6.0,
 'U': 7.0,
 'clw_zz': 8.0,
 'hus_z': 9.0,
 'zg': 10.0}

Second-order

In [105]:
return_averaged_ranking(p_folder, 'polynomial_degree_2', max_no_features=10)

{'rh': 1.0,
 'ta': 2.0,
 'clw cli': 3.0,
 'rh rh_z': 4.0,
 'ta rh_z': 5.6,
 'hus rh': 6.4,
 'ta rh': 7.4,
 'rh^2': 7.9,
 'hus_z': 9.2,
 'U': 10.1,
 'pa hus_z': 10.4,
 'zg^2': 10.5,
 'zg ta_z': 10.7,
 'zg pa_zz': 10.8}

Third-order

In [100]:
return_averaged_ranking(p_folder, 'polynomial_degree_3', max_no_features=10)

{'rh': 1.0,
 'ta': 2.0,
 'clw cli': 3.0,
 'ta^2 rh': 4.4,
 'rh^2': 5.4,
 'ta^2': 6.7,
 'rh rh_z': 7.4,
 'rh_z': 8.3,
 'pa_zz pa^2': 8.8,
 'ta rh_z': 9.4,
 'rh_z pa^2': 10.2,
 'clw rh': 10.6,
 'clw ta': 10.8}